In [ ]:
# %pip install requests beautifulsoup4

#### 1. 웹 스크래핑 환경 설정 및 유틸리티 함수
이 셀에서는 필요한 라이브러리를 임포트하고, 봇으로 인식되지 않기 위한 기본 헤더 설정 및 스크래핑 과정에서 사용할 유틸리티 함수들을 정의합니다.

In [ ]:
# 필수 라이브러리 임포트
import requests
from bs4 import BeautifulSoup
import os
import time
import random

# 봇으로 인식되지 않기 위한 헤더 설정
# 실제 브라우저 User-Agent와 Accept-Language를 사용하면 봇 감지 회피에 도움이 됩니다.
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
}

BASE_URL_FORMAT = "https://dethitiengnhat.com/en/jlpt/N{level_num}"

def format_url(level_num, year, month, type_url):
    """주어진 파라미터로 URL을 구성합니다."""
    base_url = BASE_URL_FORMAT.format(level_num=level_num)
    return f"{base_url}/{year}{month}/{type_url}"

def get_content_with_underline(element):
    """
    주어진 Beautiful Soup 태그에서 텍스트를 추출하되, <u> 태그는 유지합니다.
    이 함수는 엘리먼트 내의 모든 자식 노드를 순회하며 텍스트와 <u> 태그를 조합합니다.
    """
    if not element:
        return ""

    parts = []
    for content in element.contents: # 자식 노드를 순회
        if content.name == 'u': # <u> 태그인 경우
            parts.append(f"<u>{content.get_text(strip=True)}</u>")
        elif isinstance(content, str): # 일반 텍스트 노드인 경우
            parts.append(content.strip())
        else: # 그 외 다른 태그인 경우 (예: <span>, <div> 등), 내부 텍스트만 추출
            # 여기서는 <u> 태그 외의 다른 태그는 제거하고 텍스트만 가져오는 것으로 가정합니다.
            # 만약 다른 특정 태그도 유지해야 한다면 여기에 추가 로직을 구현해야 합니다.
            parts.append(content.get_text(strip=True))
            
    # 빈 문자열을 필터링하고 공백으로 연결합니다.
    return " ".join(filter(None, parts)).strip()


def sanitize_filename(filename):
    """파일 이름에 사용할 수 없는 문자를 제거하거나 대체합니다."""
    return "".join(c for c in filename if c.isalnum() or c in ('_', '-')).strip()

def save_to_markdown(filename, content):
    """주어진 내용을 마크다운 파일로 저장합니다."""
    # 파일명에 부적절한 문자가 포함될 경우를 대비하여 정규화합니다.
    clean_filename = sanitize_filename(filename) + ".md"
    
    # 'jlpt_data' 폴더가 없으면 생성
    output_dir = "jlpt_data"
    os.makedirs(output_dir, exist_ok=True)
    
    file_path = os.path.join(output_dir, clean_filename)
    
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"✅ 데이터 저장 완료: {file_path}")
    except IOError as e:
        print(f"❌ 파일 저장 중 오류 발생 '{file_path}': {e}")

#### 2. 스크래핑 로직 정의
이 셀에서는 주어진 URL에서 데이터를 스크래핑하고, 이를 마크다운 형식으로 구성하는 핵심 로직을 정의합니다.

In [ ]:
def scrape_page_content(url, current_type): # current_type 인자 추가

    print(f"⏳ 스크래핑 시작: {url}")
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"❌ 요청 오류 발생 ({url}): {e}")
        # 반환 값 변경: V 데이터, G 데이터, 첫 big_item_content
        return None, None, None

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 마크다운 출력을 세 가지 리스트로 분리: V 유형, G 유형
    markdown_output_V = []
    markdown_output_G = []
    markdown_output_R = []
    
    first_big_item_content = "" # 파일명 결정을 위해 첫 big_item 내용을 저장
    
    # 현재 처리 중인 마크다운 리스트를 가리키는 변수
    current_markdown_list = None 

    # 페이지 내의 모든 관련 엘리먼트를 문서 순서대로 가져옵니다.
    all_relevant_elements = soup.find_all('div', class_=['big_item', 'question_content', 'question_list', 'answer_1row', 'answer_2row'])

    if not all_relevant_elements:
        print(f"⚠️ 페이지에서 스크래핑할 요소를 찾을 수 없습니다: {url}")
        return "# 내용 없음\n", "# 내용 없음\n", ""
    
    for i, element in enumerate(all_relevant_elements):
        if 'big_item' in element.get('class', []):
            item_text = get_content_with_underline(element)
            
            # 첫 big_item 내용은 항상 저장 (파일명 결정용)
            if i == 0:
                first_big_item_content = item_text

            # current_type이 3일 때만 'G'/'R' 분리 로직 적용
            if current_type == 3:
                if any(keyword in item_text for keyword in ['問題4', '問題 4', '問題４', '問題5', '問題 5', '問題５', '問題6', '問題６', '問題 6', '問題7', '問題 7', '問題７']):
                    current_markdown_list = markdown_output_R
                    current_markdown_list.append(f"# {item_text}\n")
                else:
                    current_markdown_list = markdown_output_G
                    current_markdown_list.append(f"# {item_text}\n")
            else: # current_type이 1일 경우 (V 유형)
                current_markdown_list = markdown_output_V
                current_markdown_list.append(f"# {item_text}\n")
        
        # current_markdown_list가 설정된 경우에만 내용 추가
        elif current_markdown_list is not None:
            if 'question_content' in element.get('class', []):
                content_text = get_content_with_underline(element)
                current_markdown_list.append(f"### {content_text}\n")
                
            elif 'question_list' in element.get('class', []):
                question_text = get_content_with_underline(element)
                current_markdown_list.append(f"## {question_text}\n")
            
            elif 'answer_1row' in element.get('class', []) or 'answer_2row' in element.get('class', []):
                current_markdown_list.append("#### 答え\n") 
                
                answers = element.find_all('div', class_='answers')
                if not answers:
                    answers = element.find_all('li', class_='answers')

                for answer in answers:
                    answer_text = get_content_with_underline(answer)
                    current_markdown_list.append(f"- {answer_text}\n")
                current_markdown_list.append("\n") # 각 답변 그룹 사이에 공백 추가
    
    # 최종 반환 값 변경: V 유형 데이터, G 유형 데이터, R 유형 데이터, 첫 big_item_content
    return "\n".join(markdown_output_V), "\n".join(markdown_output_G), "\n".join(markdown_output_R), first_big_item_content


#### 3. 메인 스크래핑 실행 루프
이 셀에서는 URL 이동 규칙에 따라 반복적으로 웹페이지를 스크래핑하고, 지정된 파일명 규칙에 따라 .md 파일로 저장하는 메인 루프를 실행합니다.

In [ ]:
# 스크래핑 파라미터 초기 설정
level_num = 3
current_year = 2019
end_year = 2010
current_month = '12'
current_type = 1 # 1 또는 3

print("--- 웹 스크래핑 시작 ---")

while True:
    # URL 구성
    date_url = f"{current_year}{current_month}"
    url = format_url(level_num, current_year, current_month, current_type)

    print(f"\n🔄 현재 URL: {url}")
    
    # 페이지 스크래핑 및 데이터 반환 (반환 값이 3개로 변경됨!)
    scraped_markdown_V, scraped_markdown_G, scraped_markdown_R, first_big_item_content = scrape_page_content(url, current_type) # current_type 인자 전달
    
    if scraped_markdown_V is None: # 스크래핑 실패 시
        print(f"❗ {url} 스크래핑 실패. 다음 URL로 넘어갑니다.")
        # 실패 처리 로직 (이전과 동일)
        if current_type == 1:
            current_type = 3
        else: # current_type == 3
            if current_month == '12':
                current_month = '07'
                current_type = 1
            else: # current_month == '07'
                current_year -= 1
                current_month = '12'
                current_type = 1
        
        if current_year < end_year or (current_year == end_year and current_month == '07' and current_type == 3):
            print("🏁 모든 스크래핑 완료 조건 충족.")
            break
        
        time.sleep(random.uniform(2, 5))
        continue

    # 파일 이름 생성 및 저장 로직 변경
    if current_type == 3:
        # G 유형 데이터가 있으면 저장
        if scraped_markdown_G.strip(): # 내용이 비어있지 않은 경우
            file_name_G = f"N{level_num}_{date_url}_G"
            save_to_markdown(file_name_G, scraped_markdown_G)
        else:
            print(f"ℹ️ {url} 에서 G 유형 데이터가 없습니다.")

        # R 유형 데이터가 있으면 저장
        if scraped_markdown_R.strip(): # 내용이 비어있지 않은 경우
            file_name_R = f"N{level_num}_{date_url}_R"
            save_to_markdown(file_name_R, scraped_markdown_R)
        else:
            print(f"ℹ️ {url} 에서 G 유형 데이터가 없습니다.")

    elif current_type == 1:
        if scraped_markdown_V.strip(): # R 유형 데이터 (scraped_markdown_V에 있음)
            file_name_V = f"N{level_num}_{date_url}_V"
            save_to_markdown(file_name_V, scraped_markdown_V) # R 유형은 V 변수에 담겨 옴
        else:
            print(f"ℹ️ {url} 에서 R 유형 데이터가 없습니다.")

    # --- URL 이동 규칙 적용 (이전과 동일) ---
    if current_type == 1:
        current_type = 3
    else: # current_type == 3
        if current_month == '12':
            current_month = '07'
            current_type = 1
        else: # current_month == '07'
            current_year -= 1
            current_month = '12'
            current_type = 1
    
    # 스크래핑 종료 조건 확인 (이전과 동일)
    if current_year < end_year or (current_year == end_year and current_month == '07' and current_type == 3):
        print("🏁 모든 스크래핑 완료 조건 충족.")
        break
    
    # 봇 감지 방지를 위한 임의의 지연 시간 (이전과 동일)
    time.sleep(random.uniform(2, 5)) 

print("\n--- 웹 스크래핑 종료 ---")